In [2]:
import pandas as pd
test= pd.read_csv("/opt/ml/code/AKT/github/output/output_lgbm.csv")
test

,id,prediction
0,0,0.7194
1,1,0.8082
2,2,0.2251
3,3,0.7629
4,4,0.2121
...,...,...
739,739,0.1221
740,740,0.7481
741,741,0.8730
742,742,0.8904


In [1]:
from __future__ import absolute_import
 
import numpy
import matplotlib.pyplot as plt
import json

def plot_head_map(mma, target_labels, source_labels):
    fig, ax = plt.subplots()
    heatmap = ax.pcolor(mma, cmap=plt.cm.Blues)
 
    # put the major ticks at the middle of each cell
    ax.set_xticks(numpy.arange(mma.shape[1]) + 0.5, minor=False)
    ax.set_yticks(numpy.arange(mma.shape[0]) + 0.5, minor=False)
 
    ax.set_xlim(0, int(mma.shape[1]))
    ax.set_ylim(0, int(mma.shape[0]))
 
    # want a more natural, table-like display
    ax.invert_yaxis()
    ax.xaxis.tick_top()
 
    # source words -> column labels
    ax.set_xticklabels(source_labels, minor=False)
    # target words -> row labels
    ax.set_yticklabels(target_labels, minor=False)
 
    plt.xticks(rotation=45)
 
    # plt.tight_layout()
    plt.show()
 
 
# column labels -> target words
# row labels -> source words
 
def read_alignment_matrix(f):
    header = f.readline().strip().split('|||')
    if header[0] == '':
        return None, None, None, None
    sid = int(header[0].strip())
    # number of tokens in source and translation +1 for eos
    src_count, trg_count = map(int, header[-1].split())
    # source words
    source_labels = header[3].decode('UTF-8').split()
    # source_labels.append('</s>')
    # target words
    target_labels = header[1].decode('UTF-8').split()
    target_labels.append('</s>')
 
    mm = []
    for r in range(trg_count):
        alignment = map(float, f.readline().strip().split())
        mm.append(alignment)
    mma = numpy.array(mm)
    return sid, mma, target_labels, source_labels

    
def read_plot_alignment_matrices(f, start=0):
    attentions = json.load(f, encoding="utf-8")
 
    for idx, att in attentions.items():
        if idx < start: continue
        source_labels = att["source"].split() + ["SEQUENCE_END"]
        target_labels = att["translation"].split()
        att_list = att["attentions"]
        assert att_list[0]["type"] == "simple", "Do not use this tool for multihead attention."
        mma = numpy.array(att_list[0]["value"])
        if mma.shape[0] == len(target_labels) + 1:
            target_labels += ["SEQUENCE_END"]
 
        plot_head_map(mma, target_labels, source_labels)

In [10]:
import math
class PID_DATA(object):
    def __init__(self, n_question,  seqlen):

        self.seqlen = seqlen
        self.n_question = n_question

    def load_data(self, f_data):
        # f_data = open(path, 'r')
        q_data = []
        qa_data = []
        p_data = []
        for idx, line in enumerate(f_data):
            Q = line[1]
            P = line[2]
            A = line[3]
            # A = A[:-1]

            # start split the data
            n_split = 1
            # print('len(Q):',len(Q))
            if len(Q) > self.seqlen:
                n_split = math.floor(len(Q) / self.seqlen)
                if len(Q) % self.seqlen:
                    n_split = n_split + 1
            # print('n_split:',n_split)
            for k in range(n_split):
                question_sequence = []
                problem_sequence = []
                answer_sequence = []
                if k == n_split - 1:
                    endINdex = len(A)
                else:
                    endINdex = (k+1) * self.seqlen
                for i in range(k * self.seqlen, endINdex):
                    if Q[i]>0:
                        Xindex = int(Q[i]) + int(A[i]) * self.n_question
                        question_sequence.append(int(Q[i]))
                        problem_sequence.append(int(P[i]))
                        answer_sequence.append(Xindex)
                    else:
                        print(Q[i])
                q_data.append(question_sequence)
                qa_data.append(answer_sequence)
                p_data.append(problem_sequence)

        # f_data.close()
        ### data: [[],[],[],...] <-- set_max_seqlen is used
        # convert data into ndarrays for better speed during training
        q_dataArray = np.zeros((len(q_data), self.seqlen))
        for j in range(len(q_data)):
            dat = q_data[j]
            q_dataArray[j, :len(dat)] = dat

        qa_dataArray = np.zeros((len(qa_data), self.seqlen))
        for j in range(len(qa_data)):
            dat = qa_data[j]
            qa_dataArray[j, :len(dat)] = dat

        p_dataArray = np.zeros((len(p_data), self.seqlen))
        for j in range(len(p_data)):
            dat = p_data[j]
            p_dataArray[j, :len(dat)] = dat
        return q_dataArray, qa_dataArray, p_dataArray

In [6]:
model = load_model(args)

checkpoint = torch.load(os.path.join(
    'model', args.model, args.save, 'test') + '_'+str(20))
model.load_state_dict(checkpoint['model_state_dict'])

NameError: name 'load_model' is not defined

In [100]:
transpose_data_model = {'akt'}
import numpy as np
import torch
import math
from sklearn import metrics
from utils import model_isPid_type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def test(net, params, optimizer, q_data, qa_data, pid_data, label):
    # dataArray: [ array([[],[],..])] Shape: (3633, 200)
    pid_flag, model_type = model_isPid_type(params.model)
    net.eval()
    N = int(math.ceil(float(len(q_data)) / float(params.batch_size)))
    q_data = q_data.T  # Shape: (200,3633)
    qa_data = qa_data.T  # Shape: (200,3633)
    if pid_flag:
        pid_data = pid_data.T
    seq_num = q_data.shape[1]
    pred_list = []
    target_list = []

    count = 0
    true_el = 0
    element_count = 0
    for idx in range(N):

        q_one_seq = q_data[:, idx*params.batch_size:(idx+1)*params.batch_size]
        if pid_flag:
            pid_one_seq = pid_data[:, idx *
                                   params.batch_size:(idx+1) * params.batch_size]
        input_q = q_one_seq[:, :]  # Shape (seqlen, batch_size)
        qa_one_seq = qa_data[:, idx *
                             params.batch_size:(idx+1) * params.batch_size]
        input_qa = qa_one_seq[:, :]  # Shape (seqlen, batch_size)

        # print 'seq_num', seq_num
        if model_type in transpose_data_model:
            # Shape (seqlen, batch_size)
            input_q = np.transpose(q_one_seq[:, :])
            # Shape (seqlen, batch_size)
            input_qa = np.transpose(qa_one_seq[:, :])
            target = np.transpose(qa_one_seq[:, :])
            if pid_flag:
                input_pid = np.transpose(pid_one_seq[:, :])
        else:
            input_q = (q_one_seq[:, :])  # Shape (seqlen, batch_size)
            input_qa = (qa_one_seq[:, :])  # Shape (seqlen, batch_size)
            target = (qa_one_seq[:, :])
            if pid_flag:
                input_pid = (pid_one_seq[:, :])
        target = (target - 1) / params.n_question
        target_1 = np.floor(target)
        #target = np.random.randint(0,2, size = (target.shape[0],target.shape[1]))

        input_q = torch.from_numpy(input_q).long().to(device)
        input_qa = torch.from_numpy(input_qa).long().to(device)
        target = torch.from_numpy(target_1).float().to(device)
        if pid_flag:
            input_pid = torch.from_numpy(input_pid).long().to(device)

        with torch.no_grad():
            if pid_flag:
                loss, pred, ct = net(input_q, input_qa, target, input_pid)
            else:
                loss, pred, ct = net(input_q, input_qa, target)
        pred = pred.cpu().numpy()  # (seqlen * batch_size, 1)
        true_el += ct.cpu().numpy()
        #target = target.cpu().numpy()
        if (idx + 1) * params.batch_size > seq_num:
            real_batch_size = seq_num - idx * params.batch_size
            count += real_batch_size
        else:
            count += params.batch_size

        # correct: 1.0; wrong 0.0; padding -1.0
        target = target_1.reshape((-1,))
        nopadding_index = np.flatnonzero(target >= -0.9)
        nopadding_index = nopadding_index.tolist()
        pred_nopadding = pred[nopadding_index]
        target_nopadding = target[nopadding_index]

        element_count += pred_nopadding.shape[0]
        # print avg_loss
        pred_list.append(pred_nopadding)
        target_list.append(target_nopadding)

    assert count == seq_num, "Seq not matching"

    all_pred = np.concatenate(pred_list, axis=0)
    # all_target = np.concatenate(target_list, axis=0)
    # loss = binaryEntropy(all_target, all_pred)
    # auc = compute_auc(all_target, all_pred)
    # accuracy = compute_accuracy(all_target, all_pred)

    # return loss, accuracy, auc
    return all_pred

In [102]:
device

device(type='cuda')

In [91]:
n_q = preprocess.args.n_questions
n_q

9455

In [51]:
len(test_pid[test_qa_data<0])

744

In [94]:

dat = PID_DATA(n_question=n_q,
                       seqlen=200)
test_q_data, test_qa_data, test_pid = dat.load_data(test_data)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [53]:
from torch import nn
qa_embed_diff = nn.Embedding(2 * n_q + 1, args.embed_l)

AttributeError: 'EasyDict' object has no attribute 'embed_l'

In [98]:
test_qa_data[-1]

array([12522., 12523., 12524., 12525.,  3071., 13533., 13534., 13535.,
        4081.,  4082., 13538.,  3728., 13184., 13185., 13186., -5723.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [86]:
dat.n_question

9455

In [101]:
all_pred=test(model, args, None, test_q_data, test_qa_data, test_pid, label='Test')

RuntimeError: CUDA error: device-side assert triggered

In [103]:
all_pred=test(model, args, None, valid_q_data, valid_qa_data, valid_pid, label='Valid')

RuntimeError: CUDA error: device-side assert triggered

In [112]:
valid_pid.T.shape

(200, 5036)

In [113]:
a,b=model_isPid_type(args.model)
print(a)
print(b)

True
akt


In [127]:
N = int(math.ceil(float(len(valid_q_data)) / float(24)))
valid_q_data = valid_q_data.T  # Shape: (200,3633)
valid_qa_data = valid_qa_data.T  # Shape: (200,3633)
print(valid_q_data.shape)

(200, 5036)


In [130]:
q_one_seq = valid_q_data[:, 0:24]
q_one_seq.shape
input_q = np.transpose(q_one_seq[:, :])
input_q.shape

(24, 200)

In [134]:
input_q = np.transpose(input_q[:, :])
input_q.shape

(200, 24)

In [147]:
device = torch.device("cpu")

In [144]:
torch.from_numpy(input_q).long().unique()

tensor([   0,    1,    2,  ..., 9176, 9177, 9178])

In [148]:
input_q = torch.from_numpy(input_q).long().to(device)

In [123]:
def test(net, params, optimizer, q_data, qa_data, pid_data, label):
    # dataArray: [ array([[],[],..])] Shape: (3633, 200)
    pid_flag, model_type = model_isPid_type(params.model)
    net.eval()
    N = int(math.ceil(float(len(q_data)) / float(params.batch_size)))
    q_data = q_data.T  # Shape: (200,3633)
    qa_data = qa_data.T  # Shape: (200,3633)
    if pid_flag:
        pid_data = pid_data.T
    seq_num = q_data.shape[1]
    pred_list = []
    target_list = []

    count = 0
    true_el = 0
    element_count = 0
    for idx in range(N):

        q_one_seq = q_data[:, idx*params.batch_size:(idx+1)*params.batch_size]
        if pid_flag:
            pid_one_seq = pid_data[:, idx *
                                   params.batch_size:(idx+1) * params.batch_size]
        input_q = q_one_seq[:, :]  # Shape (seqlen, batch_size)
        qa_one_seq = qa_data[:, idx *
                             params.batch_size:(idx+1) * params.batch_size]
        input_qa = qa_one_seq[:, :]  # Shape (seqlen, batch_size)

        # print 'seq_num', seq_num
        if model_type in transpose_data_model:
            # Shape (seqlen, batch_size)
            input_q = np.transpose(q_one_seq[:, :])
            # Shape (seqlen, batch_size)
            input_qa = np.transpose(qa_one_seq[:, :])
            target = np.transpose(qa_one_seq[:, :])
            if pid_flag:
                input_pid = np.transpose(pid_one_seq[:, :])
        else:
            input_q = (q_one_seq[:, :])  # Shape (seqlen, batch_size)
            input_qa = (qa_one_seq[:, :])  # Shape (seqlen, batch_size)
            target = (qa_one_seq[:, :])
            if pid_flag:
                input_pid = (pid_one_seq[:, :])
        target = (target - 1) / params.n_question
        target_1 = np.floor(target)
        #target = np.random.randint(0,2, size = (target.shape[0],target.shape[1]))
        print(input_q.shape)
        input_q = torch.from_numpy(input_q).long().to(device)
        input_qa = torch.from_numpy(input_qa).long().to(device)
        target = torch.from_numpy(target_1).float().to(device)
        if pid_flag:
            input_pid = torch.from_numpy(input_pid).long().to(device)

        with torch.no_grad():
            if pid_flag:
                loss, pred, ct = net(input_q, input_qa, target, input_pid)
            else:
                loss, pred, ct = net(input_q, input_qa, target)
        pred = pred.cpu().numpy()  # (seqlen * batch_size, 1)
        true_el += ct.cpu().numpy()
        #target = target.cpu().numpy()
        if (idx + 1) * params.batch_size > seq_num:
            real_batch_size = seq_num - idx * params.batch_size
            count += real_batch_size
        else:
            count += params.batch_size

        # correct: 1.0; wrong 0.0; padding -1.0
        target = target_1.reshape((-1,))
        nopadding_index = np.flatnonzero(target >= -0.9)
        nopadding_index = nopadding_index.tolist()
        pred_nopadding = pred[nopadding_index]
        target_nopadding = target[nopadding_index]

        element_count += pred_nopadding.shape[0]
        # print avg_loss
        pred_list.append(pred_nopadding)
        target_list.append(target_nopadding)

    assert count == seq_num, "Seq not matching"

    all_pred = np.concatenate(pred_list, axis=0)
    all_target = np.concatenate(target_list, axis=0)
    loss = binaryEntropy(all_target, all_pred)
    auc = compute_auc(all_target, all_pred)
    accuracy = compute_accuracy(all_target, all_pred)

    return loss, accuracy, auc

In [77]:
device = torch.device("cpu")

In [151]:
args.batch_size=24
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss,acc,auc=test(model, args, None, valid_q_data, valid_qa_data, valid_pid, label='Valid')

(24, 5036)


RuntimeError: Input, output and indices must be on the current device

In [150]:
args.device='cpu'

In [1]:
def test_one_dataset(params, file_name, test_q_data, test_qa_data, test_pid,  best_epoch):
    print("\n\nStart testing ......................\n Best epoch:", best_epoch)
    model = load_model(params)

    checkpoint = torch.load(os.path.join(
        'model', params.model, params.save, file_name) + '_'+str(best_epoch))
    model.load_state_dict(checkpoint['model_state_dict'])

    test_loss, test_accuracy, test_auc = test(
        model, params, None, test_q_data, test_qa_data, test_pid, label='Test')
    print("\ntest_auc\t", test_auc)
    print("test_accuracy\t", test_accuracy)
    print("test_loss\t", test_loss)

    # Now Delete all the models
    path = os.path.join('model', params.model, params.save,  file_name) + '_*'
    for i in glob.glob(path):
        os.remove(i)

In [8]:
import easydict
import torch

data_dir = '/opt/ml/input/data/train_dataset'
file_name = 'train_all.csv'
test_file_name = 'test_data.csv'

config = {}
# 설정
config['data_dir'] = '/opt/ml/input/data/train_dataset'
config['seed'] = 42
config['device'] = "cuda" if torch.cuda.is_available() else "cpu"
config['max_iter'] = 300
config['optim'] = 'adam'
config['maxgradnorm'] = -1
config['final_fc_dim'] = 512
config['d_model'] = 256
config['d_ff'] = 1024
config['dropout'] = 0.05
config['n_block'] = 1
config['n_head'] = 8
config['kq_same'] = 1
config['l2'] = 1e-5
config['q_embed_dim'] = 50
config['qa_embed_dim'] = 256
config['memory_size'] = 50
config['init_std'] = 0.1
config['hidden_dim'] = 512
config['lamda_r'] = 0.1
config['lamda_w1'] = 0.1
config['lamda_w2'] = 0.1
config['model'] = 'akt_pid'
config['n_question'] = preprocess.args.n_questions
config['n_pid'] = preprocess.args.n_tag
config['seqlen'] = 200
config['save'] = 'asset'
config['load'] = 'asset'
config['asset_dir'] = 'asset'
config['model_dir'] = 'models'
config['model_name'] = 'model.pt'
config['output_dir'] = 'output'
    
args = easydict.EasyDict(config)

In [7]:
setSeeds(42)

preprocess = Preprocess(args)
preprocess.load_test_data(test_file_name)

test_data = preprocess.get_test_data()
# train_data, valid_data = preprocess.split_data(train_data)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:61: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [39]:
n_q = preprocess.args.n_questions
n_q

9455

In [48]:
N = int(math.ceil(float(len(test_q_data)) / float(24)))
N

72

In [51]:
from utils import model_isPid_type
def test(net, params, optimizer, q_data, qa_data, pid_data, label):
    # dataArray: [ array([[],[],..])] Shape: (3633, 200)
    pid_flag, model_type = model_isPid_type(params.model)
    net.eval()
    N = int(math.ceil(float(len(q_data)) / float(params.batch_size)))
    q_data = q_data.T  # Shape: (200,3633)
    qa_data = qa_data.T  # Shape: (200,3633)
    if pid_flag:
        pid_data = pid_data.T
    seq_num = q_data.shape[1]
    pred_list = []
    target_list = []

    count = 0
    true_el = 0
    element_count = 0
    for idx in range(N):

        q_one_seq = q_data[:, idx*params.batch_size:(idx+1)*params.batch_size]
        if pid_flag:
            pid_one_seq = pid_data[:, idx *
                                   params.batch_size:(idx+1) * params.batch_size]
        input_q = q_one_seq[:, :]  # Shape (seqlen, batch_size)
        qa_one_seq = qa_data[:, idx *
                             params.batch_size:(idx+1) * params.batch_size]
        input_qa = qa_one_seq[:, :]  # Shape (seqlen, batch_size)

        # print 'seq_num', seq_num
        if model_type in transpose_data_model:
            # Shape (seqlen, batch_size)
            input_q = np.transpose(q_one_seq[:, :])
            # Shape (seqlen, batch_size)
            input_qa = np.transpose(qa_one_seq[:, :])
            target = np.transpose(qa_one_seq[:, :])
            if pid_flag:
                input_pid = np.transpose(pid_one_seq[:, :])
        else:
            input_q = (q_one_seq[:, :])  # Shape (seqlen, batch_size)
            input_qa = (qa_one_seq[:, :])  # Shape (seqlen, batch_size)
            target = (qa_one_seq[:, :])
            if pid_flag:
                input_pid = (pid_one_seq[:, :])
        target = (target - 1) / params.n_question
        target_1 = np.floor(target)
        #target = np.random.randint(0,2, size = (target.shape[0],target.shape[1]))
        print(input_q.shape)
        input_q = torch.from_numpy(input_q).long().to(device)
        input_qa = torch.from_numpy(input_qa).long().to(device)
        target = torch.from_numpy(target_1).float().to(device)
        if pid_flag:
            input_pid = torch.from_numpy(input_pid).long().to(device)

        with torch.no_grad():
            if pid_flag:
                loss, pred, ct = net(input_q, input_qa, target, input_pid)
            else:
                loss, pred, ct = net(input_q, input_qa, target)
        pred = pred.cpu().numpy()  # (seqlen * batch_size, 1)
        true_el += ct.cpu().numpy()
        #target = target.cpu().numpy()
        if (idx + 1) * params.batch_size > seq_num:
            real_batch_size = seq_num - idx * params.batch_size
            count += real_batch_size
        else:
            count += params.batch_size

        # correct: 1.0; wrong 0.0; padding -1.0
        target = target_1.reshape((-1,))
        # nopadding_index = np.flatnonzero(target >= -0.9)
        nopadding_index = np.flatnonzero(target == -1)
        nopadding_index = nopadding_index.tolist()
        pred_nopadding = pred[nopadding_index]
        # pred_nopadding = pred
        # target_nopadding = target[nopadding_index]

        # element_count += pred_nopadding.shape[0]
        # print avg_loss
        pred_list.append(pred_nopadding)
        # target_list.append(target_nopadding)

    assert count == seq_num, "Seq not matching"


    return pred_list

In [64]:
test_qa.size

344400

In [66]:
test_qa.shape

(1722, 200)

In [75]:
len(test_qa<0)

1722

In [63]:
pred_test=np.reshape(pred, (1722, 200))
len(pred_test[0])

ValueError: cannot reshape array of size 72 into shape (1722,200)

In [71]:
args.batch_size=1
# transpose_data_model={'akt'}
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pred= test(model, args, None, test_q_data, test_qa, test_pid, label='test')

(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(

In [57]:
len(pred[1])

4800

In [76]:
all_pred = np.concatenate(pred, axis=0)
all_pred.size

344400

In [83]:
preds=all_pred[all_qa<0]
len(preds)

744

In [84]:
write_path = os.path.join(args.output_dir, "output_akt.csv")
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)    
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/output_akt.csv


In [77]:
all_qa = np.concatenate(test_qa_data, axis=0)
all_qa.size

344400

In [50]:
test_qa[0]

array([13690., 13691.,  4237.,  4238.,  4240.,  4241.,  4239.,  4263.,
        4264.,  4265.,  4266.,  4268.,  4269.,  4267., 13781.,  4327.,
        4328., 13784.,  4330.,  4331.,  4332., 13795., 13796., 13797.,
       13798., 13799.,  4345.,  4346., 13840., 13841.,  4387.,  4388.,
       13844.,  4390.,  4391., 10556., 10557., 10558., 10559., 10560.,
       10561., 10562., 10563., 10564., 10565., 10566., 10567., 10568.,
       10569., 10570., 10571., 10572., 10573., 10574., 10575., 10576.,
       10577., 10547., 10548., 10549., 10550., 10551., 10552., 10553.,
       10578., 10579.,  1125., 10581., 10582., 10583., 10584., 10585.,
       10586., 10587., 10588., 10589., 10590., 10591., 10592., 10593.,
       10594., 10595., 10596., 10597.,  1143., 10599., 10600., 10601.,
       10608.,  1154., 10610., 10611., 10612., 10613., 10614., 10615.,
       10554., 10555., 10602., 10603., 10604., 10605., 10606., 10607.,
        4660., 14116., 14117., 14118.,  4664.,  4665., 14121., 10712.,
      

In [24]:
len(test_qa_data[test_qa_data<0])

744

In [13]:
test_qa=test_qa_data.copy()

In [14]:
test_qa[test_qa<0]=0

In [12]:
from utils import load_model
model = load_model(args)

checkpoint = torch.load(os.path.join(
    'model', args.model, args.save, 'test') + '_'+str(20))
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [11]:
dat = PID_DATA(n_question=args.n_question,
                       seqlen=200)
test_q_data, test_qa_data, test_pid = dat.load_data(test_data)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [64]:
test_qa_data[5]

array([14382.,  4928.,  4929., 14499.,  5045.,  5046., 14502.,  5048.,
        5049., 14392., 14393., 14394., 14395., 14396., 14397.,  4943.,
       14406., 14407., 14408., 14409., 14410., 14411.,  4957.,  5055.,
        5056., 14512., 14513.,  5059., 14413., 14414., 14415., 14416.,
       14417., 14418.,  4964., -4490.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [37]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tt=test_q_data[test_qa_data<0]
# tt = torch.Tensor([tt]).long().to
tt = torch.Tensor(tt).long().to(device)
tt.shape

torch.Size([744])

In [38]:
tt_a = torch.Tensor([1]).long().to(device)
tt_a

tensor([1], device='cuda:0')

In [20]:
tt_q=model.q_embed(tt)

In [39]:
tt_a=model.qa_embed(tt_a)
tt_a

tensor([[ 1.1452e+00,  3.0688e-01, -2.9829e-02,  1.8261e-01, -3.2002e-01,
          1.4787e+00,  2.0356e-01, -5.9621e-02, -4.5312e-01, -1.3840e+00,
         -4.3169e-01, -2.9451e-01,  5.3537e-01,  1.3464e+00,  4.9898e-01,
         -3.5237e-02,  6.8981e-01, -8.3332e-01,  2.5560e+00, -8.1424e-01,
          2.7735e-01,  1.5562e+00, -6.8949e-01, -1.3928e+00,  8.3668e-01,
          8.1764e-01, -4.0723e-01, -1.8605e+00, -1.7553e-01, -1.0286e+00,
         -7.6797e-02,  2.4325e-01,  1.8028e+00,  7.5578e-02, -2.1831e-01,
          1.6655e+00,  5.3284e-01, -3.1665e-01, -1.1740e+00,  4.3087e-01,
         -7.7086e-01, -5.7961e-02,  2.1224e+00,  1.8020e+00, -1.5182e+00,
         -4.1975e-01,  8.3604e-01, -2.2276e-01,  1.3628e+00, -1.0267e+00,
         -3.3248e-01,  8.7700e-01,  6.6532e-02,  1.1426e-01, -1.9450e+00,
         -1.7767e+00, -8.6300e-01,  4.9451e-01, -2.2454e-01, -1.1896e+00,
         -1.5917e+00, -1.0079e+00,  1.4060e-01, -5.9011e-01, -1.4436e+00,
          4.3004e-01, -7.3533e-01, -1.

In [40]:
qd_e = tt_a+tt_q
qd_e

tensor([[ 1.3594,  0.8826,  0.0990, -0.3442,  1.2487,  0.3218,  0.3483, -2.0587,
         -1.4785, -2.2603,  0.5728, -1.8763, -0.3408,  0.3688,  1.8798, -0.0999,
          1.5773, -1.9733,  1.4121, -0.4116,  2.8655,  1.0827,  0.2158, -1.4430,
          1.0437,  0.6388, -0.3672, -2.9559,  0.0342, -0.4989,  1.0901, -0.0645,
          0.0991,  0.4395, -0.9600,  0.2335,  0.2170, -0.9479,  0.2520,  0.8002,
         -2.8480,  2.1689,  1.0637,  2.0842, -1.1276, -0.3783, -0.6248, -0.2147,
         -1.4248, -0.5662, -0.7844,  0.6727,  0.6760,  0.7424, -3.7587, -1.4366,
         -1.0418,  1.7875, -0.9385, -1.7401, -0.9464,  0.2496,  1.8283,  0.1886,
         -0.8977, -0.8022, -0.3813, -0.1638, -0.6861,  0.9227, -0.4341,  1.8425,
          1.1497, -0.2327,  1.8249,  0.3363,  0.2218,  0.3061, -2.4267,  1.9131,
         -0.3945,  0.6016,  0.9345,  0.4843,  2.1649,  0.8584,  0.9726, -2.0890,
         -0.4740, -2.8207,  3.0228,  1.4978, -0.5354,  0.0747,  0.6260,  0.2721,
          1.2077,  1.0636,  

In [28]:
# tt_d = model.q_embed_diff(tt)
qd_e.view(-1,1,256).shape

torch.Size([1, 1, 256])

In [41]:
test_out = model.model(tt_q.view(-1,1,256),qd_e.view(-1,1,256))

In [42]:
concat_q = torch.cat([test_out, tt_q.view(-1,1,256)], dim=-1)

In [43]:
output = model.out(concat_q)
output.reshape(-1)
output

tensor([[[-0.1894]]], device='cuda:0', grad_fn=<AddBackward0>)

In [44]:
from torch import nn
m = nn.Sigmoid()

In [45]:
m(output)

tensor([[[0.4528]]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [71]:
tt=test_qa_data[test_qa_data<0][0]
tt = torch.Tensor([tt]).long().to(device)
tt

tensor([-4490], device='cuda:0')

In [ ]:
model.model(tt_q,qd_e)

In [54]:
model

AKT(
  (difficult_param): Embedding(914, 1)
  (q_embed_diff): Embedding(9456, 256)
  (qa_embed_diff): Embedding(18911, 256)
  (q_embed): Embedding(9456, 256)
  (qa_embed): Embedding(2, 256)
  (model): Architecture(
    (blocks_1): ModuleList(
      (0): TransformerLayer(
        (masked_attn_head): MultiHeadAttention(
          (v_linear): Linear(in_features=256, out_features=256, bias=True)
          (k_linear): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.05, inplace=False)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
        )
        (layer_norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.05, inplace=False)
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (activation): ReLU()
        (dropout): Dropout(p=0.05, inplace=False)
        (linear2): Linear(in_features=2048, out_features=256, bias=True)
        (layer_norm2): LayerNorm((256,),

In [29]:
import numpy as np
import torch
import math
from sklearn import metrics
from utils import model_isPid_type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transpose_data_model = {'akt'}
test_loss, test_accuracy, test_auc, all_pred = test(
        model, args, None, test_q_data, test_qa_data, test_pid, label='Test')

In [67]:
all_pred = test(
        model, args, None, test_q_data, test_qa_data, test_pid, label='Test')

NameError: name 'model_isPid_type' is not defined

In [27]:
def binaryEntropy(target, pred, mod="avg"):
    loss = target * np.log(np.maximum(1e-10, pred)) + \
        (1.0 - target) * np.log(np.maximum(1e-10, 1.0-pred))
    if mod == 'avg':
        return np.average(loss)*(-1.0)
    elif mod == 'sum':
        return - loss.sum()
    else:
        assert False


def compute_auc(all_target, all_pred):
    #fpr, tpr, thresholds = metrics.roc_curve(all_target, all_pred, pos_label=1.0)
    return metrics.roc_auc_score(all_target, all_pred)


def compute_accuracy(all_target, all_pred):
    all_pred[all_pred > 0.5] = 1.0
    all_pred[all_pred <= 0.5] = 0.0
    return metrics.accuracy_score(all_target, all_pred)


In [51]:
# x1 = torch.arange(200)
x1 = torch.arange(200).expand(200, -1)
x1

tensor([[  0,   1,   2,  ..., 197, 198, 199],
        [  0,   1,   2,  ..., 197, 198, 199],
        [  0,   1,   2,  ..., 197, 198, 199],
        ...,
        [  0,   1,   2,  ..., 197, 198, 199],
        [  0,   1,   2,  ..., 197, 198, 199],
        [  0,   1,   2,  ..., 197, 198, 199]])

In [52]:
x2=x1.transpose(0,1)
x2

tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  2,   2,   2,  ...,   2,   2,   2],
        ...,
        [197, 197, 197,  ..., 197, 197, 197],
        [198, 198, 198,  ..., 198, 198, 198],
        [199, 199, 199,  ..., 199, 199, 199]])

In [53]:
x3=x2.contiguous()
x3

tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  2,   2,   2,  ...,   2,   2,   2],
        ...,
        [197, 197, 197,  ..., 197, 197, 197],
        [198, 198, 198,  ..., 198, 198, 198],
        [199, 199, 199,  ..., 199, 199, 199]])

In [56]:
model.model.blocks_1[0]

TransformerLayer(
  (masked_attn_head): MultiHeadAttention(
    (v_linear): Linear(in_features=256, out_features=256, bias=True)
    (k_linear): Linear(in_features=256, out_features=256, bias=True)
    (dropout): Dropout(p=0.05, inplace=False)
    (out_proj): Linear(in_features=256, out_features=256, bias=True)
  )
  (layer_norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.05, inplace=False)
  (linear1): Linear(in_features=256, out_features=2048, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.05, inplace=False)
  (linear2): Linear(in_features=2048, out_features=256, bias=True)
  (layer_norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (dropout2): Dropout(p=0.05, inplace=False)
)